# Попробуем выжать все из простых фичей

# Загрузка данных

In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
import pickle

In [3]:
%%time
pd_train = pickle.load(open("../tdata/pd_train.pickle", "rb"))
pd_test = pickle.load(open("../tdata/pd_test.pickle", "rb"))
pd_data = pd.concat([pd_train, pd_test], axis=0, keys=["train", "test"])
del pd_train, pd_test

CPU times: user 6.48 s, sys: 2.15 s, total: 8.63 s
Wall time: 9.43 s


## Функции

In [25]:
from sklearn.feature_extraction.text import CountVectorizer

In [6]:
pd_data.sort_values("$date", inplace = True)

In [29]:
lb_author = CountVectorizer(min_df = 1 / 26739)

In [30]:
lb_author.fit(pd_data.nickname.fillna("#empty#").values)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None,
        min_df=3.739855641572235e-05, ngram_range=(1, 1),
        preprocessor=None, stop_words=None, strip_accents=None,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, vocabulary=None)

In [31]:
len(lb_author.vocabulary_)

3696

In [36]:
X = lb_author.transform(pd_data.nickname.fillna("#empty#").values).toarray()

In [37]:
X = X.astype("int16")

In [38]:
X.shape

(172913, 3696)

In [39]:
np_author = (X.cumsum(axis=0) * X).sum(axis=1)

In [40]:
np_author

array([   1,    2,    3, ...,    0, 4979,    0])

In [41]:
np_author_clean = np.where(pd_data["nickname"].isnull(), 0, np_author)

In [44]:
np_author_clean

array([   1,    2,    3, ...,    0, 4979,    0])

In [47]:
pd_data["author_rating"] = np_author_clean

In [53]:
pd_data.sort_index(inplace=True)

In [56]:
pd_data["author_rating"].to_pickle("../features/author_rating.pickle")